# Introduction

In a prior notebook, documents were partitioned by assigning them to the domain with the highest Dice similarity of their term and structure occurrences. Here, we'll assess whether the observed modularity of the partitions is greater than expected by chance. Modularity will be measured by the ratio of dispersion between partitions to dispersion within partitions.

# Load the data

In [1]:
import pandas as pd
import numpy as np
from utilities import *

In [2]:
framework = "rdoc" 
version = 190124 # Document-term matrix version
suffix = "_opsim" # Suffix for term lists
n_iter = 10000 # Iterations for bootstrap and null
dx = [0.38, 0.38, 0.38, 0.35, 0.36, 0.36, 0.35] # Nudges for plotted means

## Brain activation coordinates

In [3]:
act_bin = load_coordinates()
print("Document N={}, Structure N={}".format(
      act_bin.shape[0], act_bin.shape[1]))

Document N=18155, Structure N=114


## Document-term matrix

In [ ]:
dtm_bin = load_doc_term_matrix(version=version, binarize=True)
print("Document N={}, Term N={}".format(
      dtm_bin.shape[0], dtm_bin.shape[1]))

## Domain archetypes

In [ ]:
from collections import OrderedDict

In [ ]:
lists, circuits = load_framework(framework, suffix=suffix)

In [ ]:
words = sorted(list(set(lists["TOKEN"])))
structures = sorted(list(set(act_bin.columns)))
domains = list(OrderedDict.fromkeys(lists["DOMAIN"]))

In [ ]:
archetypes = pd.DataFrame(0.0, index=words+structures, columns=domains)
for dom in domains:
    for word in lists.loc[lists["DOMAIN"] == dom, "TOKEN"]:
        archetypes.loc[word, dom] = 1.0
    for struct in structures:
        archetypes.loc[struct, dom] = circuits.loc[struct, dom]
archetypes[archetypes > 0.0] = 1.0
print("Term & Structure N={}, Domain N={}".format(
      archetypes.shape[0], archetypes.shape[1]))

## Document structure-term vectors

In [ ]:
pmids = dtm_bin.index.intersection(act_bin.index)
len(pmids)

In [ ]:
dtm_words = dtm_bin.loc[pmids, words]
act_structs = act_bin.loc[pmids, structures]

In [ ]:
docs = dtm_words.copy()
docs[structures] = act_structs.copy()
docs.head()

## Document assignments

In [ ]:
doc2dom_df = pd.read_csv("../partition/data/doc2dom_{}.csv".format(framework), 
                         header=None, index_col=0)
doc2dom = {int(pmid): int(dom) for pmid, dom in doc2dom_df.iterrows()}

In [ ]:
dom2docs = {dom: [] for dom in domains}
for doc, dom in doc2dom.iteritems():
    dom2docs[domains[dom-1]].append(doc)

# Compute domain modularity

In [ ]:
from scipy.spatial.distance import cdist

## Observed values

In [ ]:
centroid = np.mean(docs).values
df_obs = pd.DataFrame(index=domains, columns=pmids)
df = pd.DataFrame(index=domains, columns=["OBSERVED"])
for dom in domains:
    dom_pmids = dom2docs[dom]
    dom_vecs = docs.loc[dom_pmids].values
    dom_centroid = np.mean(dom_vecs, axis=0)
    dist_ext = (dom_vecs - centroid) ** 2
    dist_int = (dom_vecs - dom_centroid) ** 2
    df_obs.loc[dom, dom_pmids] = np.sum(dist_ext, axis=1) / np.sum(dist_int, axis=1)
    df.loc[dom, "OBSERVED"] = np.mean(np.sum(dist_ext) / np.sum(dist_int))
df.head(8)

## Null distributions

In [ ]:
import os

In [ ]:
file_null = "data/mod_null_{}_{}iter.csv".format(framework, n_iter)
if not os.path.isfile(file_null):
    df_null = np.empty((len(domains), n_iter))
    for n in range(n_iter):
        null = np.random.choice(range(docs.shape[1]), size=docs.shape[1], replace=False)
        centroid = np.mean(docs)[null].values
        for i, dom in enumerate(domains):
            dom_pmids = dom2docs[dom]
            dom_vecs = docs.loc[dom_pmids].values
            dom_centroid = np.mean(dom_vecs, axis=0)[null]
            n_docs = dom_vecs.shape[0]
            dist_ext = (dom_vecs - centroid) ** 2
            dist_int = (dom_vecs - dom_centroid) ** 2
            df_null[i,n] = np.sum(dist_ext) / np.sum(dist_int)
        if n % int(n_iter / 10.0) == 0:
            print("Processed {} iterations".format(n))
    df_null = pd.DataFrame(df_null, index=domains, columns=range(n_iter))
    df_null.to_csv(file_null)
else:
    df_null = pd.read_csv(file_null, index_col=0, header=0)

# Perform significance testing

In [ ]:
from statsmodels.stats import multitest

In [ ]:
alphas = [0.0001, 0.00001, 0.000001]

In [ ]:
pval = []
for dom in domains:   
    dom_null = df_null.loc[dom].values
    dom_obs = float(df.loc[dom, "OBSERVED"])
    p = np.sum(np.greater(dom_null, dom_obs)) / float(n_iter)
    pval.append(p)
    df.loc[dom, "P"] = p
df["FDR"] = multitest.multipletests(pval, method="fdr_bh")[1]
for dom in domains:
    for star, alpha in zip(["*", "**", "***"], alphas):
        if df.loc[dom, "FDR"] < alpha:
            df.loc[dom, "STARS"] = star
        else:
            df.loc[dom, "STARS"] = ""
df = df.loc[domains, ["OBSERVED", "P", "FDR", "STARS"]]
df

# Plot results

In [ ]:
%matplotlib inline

In [ ]:
palette = palettes[framework]

In [ ]:
plot_violins(framework, domains, df, df_null, df_obs, palette, 
             dx=dx, alphas=alphas, interval=0.999)